In [476]:
from sklearn.svm import SVC
from os import listdir
from os.path import isfile, join
import numpy as np
import cv2
import pandas as pd

In [477]:
mypath = 'C:/Users/user/Desktop/Minor/PETA/PETA dataset/3DPeS/archive'
#onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath,f))]
k = listdir(mypath)
images = np.empty((len(k)), dtype=object)
for n in range(0, len(k)-1):
    images[n] = cv2.imread(join(mypath,k[n]))

In [478]:
labels = k[-1]

In [479]:
data = pd.read_csv(join(mypath,k[-1]), sep=" ", header = None, error_bad_lines = False)
m,n = data.shape
data.set_index([0],inplace=True)
data.sort_index(0,inplace=True)
data['Upper Body'] = np.ones((m,1))
a = data.columns.tolist()
data.columns = ['Upper Body Color','Lower Body Color', 'Hair Color', 'Footwear Color', 'Pants:Casual/Formal',
                'Pants', 'Age', 'Sex', 'Casual/Formal','Sleeve Length','Shirt Design','Shirt Type','Hair Length',
                'Footwear','Carrying','Accessory','Upper Body']
a = data.columns.tolist()
i = a.index('Upper Body')
j = a.index('Lower Body Color')
a = a[:j] + a[i:i+1] + a[j:-1]
data = data[a]
#Original DataFrame has 174 rows

b'Skipping line 13: expected 17 fields, saw 18\nSkipping line 16: expected 17 fields, saw 18\nSkipping line 21: expected 17 fields, saw 19\nSkipping line 28: expected 17 fields, saw 18\nSkipping line 31: expected 17 fields, saw 18\nSkipping line 48: expected 17 fields, saw 18\nSkipping line 54: expected 17 fields, saw 18\nSkipping line 57: expected 17 fields, saw 18\nSkipping line 64: expected 17 fields, saw 18\nSkipping line 68: expected 17 fields, saw 18\nSkipping line 93: expected 17 fields, saw 18\nSkipping line 103: expected 17 fields, saw 18\nSkipping line 124: expected 17 fields, saw 18\nSkipping line 136: expected 17 fields, saw 18\nSkipping line 142: expected 17 fields, saw 19\nSkipping line 156: expected 17 fields, saw 18\nSkipping line 166: expected 17 fields, saw 18\nSkipping line 179: expected 17 fields, saw 19\nSkipping line 188: expected 17 fields, saw 18\n'


In [495]:
#Collecting upperBody from Lower Body Color
pd.set_option('max_colwidth',1000)
data['Upper Body'][data['Lower Body Color'].str.startswith('upperBody')] = ...
data['Lower Body Color'][data['Lower Body Color'].str.startswith('upperBody')]
pd.set_option("max_rows",200)
pd.set_option("large_repr",'info')
#At this point, we are done with the first two columns (upperBody values). They don't have to be touched now, except in
#later stages

In [494]:
data['Upper Body']

0
1             1
2             1
3             1
4      Ellipsis
5      Ellipsis
6             1
7             1
8      Ellipsis
9             1
10     Ellipsis
11            1
12            1
13            1
14            1
15     Ellipsis
16            1
17            1
18            1
19            1
20            1
22     Ellipsis
23            1
25            1
26            1
27            1
28            1
29            1
30            1
31            1
32            1
33            1
34            1
35            1
36            1
37            1
38            1
39            1
42            1
43            1
45            1
46     Ellipsis
47            1
48            1
49            1
50            1
51            1
52            1
53            1
54            1
55            1
56            1
57            1
58            1
59            1
61            1
62            1
63            1
64            1
65            1
66            1
68            1
70            1
71    

In [431]:
#Collecting lowerBody from Lower Body color Hair Color column
data['Lower Body Color'][data['Hair Color'].str.startswith('lowerBody')] = data['Hair Color'][data['Hair Color'].str.startswith('lowerBody')]
#We have collected all lowerBody color information in the same column

In [432]:
#Collecting hair from Footwear Color and lower body color
data['Hair Color'][data['Footwear Color'].str.startswith('hair')] =...
data['Footwear Color'][data['Footwear Color'].str.startswith('hair')]
data['Hair Color'][data['Lower Body Color'].str.startswith('hair')] =...
data['Lower Body Color'][data['Lower Body Color'].str.startswith('hair')]
#data['Hair Color'].value_counts()
#Done with hair color. 'footwear' and 'lower body' entries have to be omitted later

Series([], Name: Lower Body Color, dtype: object)

In [433]:
#Collecting footwear from hair and pants
data['Footwear Color'][data['Hair Color'].str.startswith('footwear',na=False)] =...
data['Hair Color'][data['Hair Color'].str.startswith('footwear',na=False)]
data['Footwear Color'][data['Pants:Casual/Formal'].str.startswith('footwear',na=False)] =...
data['Pants:Casual/Formal'][data['Pants:Casual/Formal'].str.startswith('footwear',na=False)]
#Done with footwear

0
4      footwearBlack
8      footwearBrown
10     footwearBlack
14     footwearWhite
15      footwearGrey
17     footwearWhite
19     footwearWhite
22     footwearBlack
46     footwearBlack
50     footwearWhite
80     footwearBlack
106    footwearBlack
113    footwearWhite
117    footwearWhite
129    footwearWhite
132    footwearWhite
137    footwearWhite
140    footwearWhite
142    footwearWhite
150    footwearBrown
164    footwearWhite
167    footwearBlack
171    footwearWhite
184    footwearBlack
186    footwearBlack
193    footwearBlack
Name: Pants:Casual/Formal, dtype: object

In [434]:
#Collecting pants
data['Pants'][data['Age'].str.startswith('lower')]=data['Age'][data['Age'].str.startswith('lower',na=False)]
mask = (data['Pants:Casual/Formal'].str.contains('lowerBodyCasual')|data['Pants:Casual/Formal'].str.contains('lowerBodyFormal'))&~(data['Pants'].str.contains('lowerBody'))
data['Pants'][mask]=data['Pants:Casual/Formal'][mask]
#Done with Pants. We are not using the 'Casual/Formal' column but we're incorporating those values in the 'Pants' column wherever
#possible

In [435]:
#Collecting Age
data['Age'][data['Pants'].str.contains('personalL',na=False)] = data['Pants'][data['Pants'].str.contains('personalL',na=False)]
data['Age'][data['Sex'].str.contains('personalL')] = data['Sex'][data['Sex'].str.contains('personalL')]
#Done with age

In [436]:
#Collecting Sex
mask2 = data['Casual/Formal'].str.contains('personalMale',na=False)
data['Sex'][mask2] = data['Casual/Formal'][mask2]
data['Sex'].value_counts()
#Done with Sex

personalMale       151
personalLess30      13
personalFemale       4
upperBodyFormal      2
personalLess45       2
upperBodyLogo        1
upperBodyCasual      1
Name: Sex, dtype: int64

In [437]:
#Collecting Sleeve Length
mask1 = data['Casual/Formal'].str.contains('Sleeve')
data['Sleeve Length'][mask1] = data['Casual/Formal'][mask1]
mask2 = data['Shirt Design'].str.contains('Sleeve')
data['Sleeve Length'][mask2] = data['Shirt Design'][mask2]
data['Sleeve Length'].value_counts()
#Done with sleeves

upperBodyShortSleeve    111
upperBodyLongSleeve      52
upperBodyCasual           5
upperBodyNoSleeve         3
upperBodyOther            1
upperBodyTshirt           1
upperBodySweater          1
Name: Sleeve Length, dtype: int64

In [438]:
#We're skipping shirt design and shirt type and moving on to Hair Length
mask1 = data['Shirt Type'].str.contains('hair',na=False)
mask2 = data['Footwear'].str.contains('hair',na=False)
data['Hair Length'][mask1] = data['Shirt Type'][mask1]

data['Hair Length'][mask2] = data['Footwear'][mask2]
#Moving on

In [439]:
#Footwear
mask1 = data['Carrying'].str.contains('footwear',na=False)
mask2 = data['Hair Length'].str.contains('footwear',na=False)
data['Footwear'][mask1] = data['Carrying'][mask1]
data['Footwear'][mask2] = data['Hair Length'][mask2]

In [440]:
#Carrying
mask1 = data['Footwear'].str.contains('carrying',na=False)
mask2 = data['Accessory'].str.contains('carrying',na=False)
mask3 = data['Hair Length'].str.contains('carrying',na=False)
data['Carrying'][mask1] = data['Footwear'][mask1]
data['Carrying'][mask2] = data['Accessory'][mask2]
data['Carrying'][mask3] = data['Hair Length'][mask3]

In [441]:
#Accesory is already taken care of
mask1 = data['Carrying'].str.contains('accessory',na=False)
mask2 = data['Footwear'].str.contains('accessory',na=False)
data['Accessory'][mask1] = data['Carrying'][mask1]
data['Accessory'][mask2] = data['Footwear'][mask2]

In [442]:
data = data.drop(['Pants:Casual/Formal','Casual/Formal','Shirt Type','Shirt Design'],axis=1)
data.head(5)

,Upper Body Color,Upper Body,Lower Body Color,Hair Color,Footwear Color,Pants,Age,Sex,Sleeve Length,Hair Length,Footwear,Carrying,Accessory
0,,,,,,,,,,,,,
1,upperBodyBlue,1,lowerBodyBlack,hairBlack,footwearWhite,lowerBodyJeans,personalLess45,personalMale,upperBodyShortSleeve,hairShort,footwearShoes,carryingMessengerBag,accessoryNothing
2,upperBodyRed,1,lowerBodyGrey,hairBrown,footwearWhite,lowerBodyJeans,personalLess30,personalLess30,upperBodyShortSleeve,hairShort,carryingNothing,carryingNothing,NaN
3,upperBodyBlue,1,lowerBodyBlack,hairBlack,footwearWhite,lowerBodyTrousers,personalLess30,personalMale,upperBodyShortSleeve,hairShort,carryingNothing,carryingNothing,NaN
4,upperBodyBlack,Ellipsis,lowerBodyBlack,Ellipsis,Ellipsis,lowerBodyJeans,personalLess30,personalMale,upperBodyShortSleeve,hairShort,carryingNothing,carryingNothing,NaN
5,upperBodyBlack,Ellipsis,lowerBodyBlue,lowerBodyBlue,footwearBrown,lowerBodyJeans,personalLarger60,personalMale,upperBodyShortSleeve,carryingNothing,accessoryNothing,carryingNothing,accessoryNothing


In [443]:
from IPython.core.display import HTML
css = open(r'C:\Users\user\Desktop\Programming\pycon-pandas-tutorial-1.0\style-table.css').read() + open(r'C:\Users\user\Desktop\Programming\pycon-pandas-tutorial-1.0\style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [444]:
mask = data['Hair Color'].str.startswith('hair') | data['Hair Color'].str.contains('Ellipsis')
data['Hair Color'][~mask] = " "
data['Hair Color'].value_counts()

hairBlack     133
               18
hairBrown      16
hairGrey        4
hairYellow      2
hairWhite       1
Name: Hair Color, dtype: int64

In [445]:
mask = data['Footwear Color'].str.startswith('footwear') | data['Hair Color'].str.contains('Ellipsis')
data['Footwear Color'][~mask] = " "
data['Footwear Color'].value_counts()

footwearWhite    63
footwearBlack    58
                 31
footwearBrown    19
footwearGrey      3
Name: Footwear Color, dtype: int64

In [446]:
mask = data['Pants'].str.startswith('lower',na=False)
data['Pants'][~mask] = " "
data['Pants'].value_counts()

lowerBodyJeans         69
lowerBodyTrousers      50
lowerBodySuits         24
lowerBodyShorts        24
                        4
lowerBodyShortSkirt     2
lowerBodyCasual         1
Name: Pants, dtype: int64

In [447]:
mask = data['Age'].str.startswith('personal',na=False)
data['Age'][~mask] = " "
data['Age'].value_counts()

personalLess30      96
personalLess45      52
personalLess60      15
personalLarger60     5
                     5
personalLess15       1
Name: Age, dtype: int64

In [448]:
mask = data['Sex'].str.contains('Male',na=False) | data['Sex'].str.contains('Female')
data['Sex'][~mask] = " "
data['Sex'].value_counts()

personalMale      151
                   19
personalFemale      4
Name: Sex, dtype: int64

In [449]:
mask = data['Sleeve Length'].str.contains('Sleeve',na=False)
data['Sleeve Length'][~mask] = " "
data['Sleeve Length'].value_counts()

upperBodyShortSleeve    111
upperBodyLongSleeve      52
                          8
upperBodyNoSleeve         3
Name: Sleeve Length, dtype: int64

In [451]:
mask = data['Hair Length'].str.startswith('hair',na=False)
data['Hair Length'][~mask] = " "
data['Hair Length'].value_counts()

hairShort    117
              40
hairLong      13
hairBald       4
Name: Hair Length, dtype: int64

In [452]:
#We'll drop footwear because there are not enough data points for this column
data = data.drop(['Footwear'],axis=1)

In [453]:
#Cleaning the Upper Body Column
mask = (data['Upper Body'] == 1)
data['Upper Body'][mask] = " "
data.head(5)

,Upper Body Color,Upper Body,Lower Body Color,Hair Color,Footwear Color,Pants,Age,Sex,Sleeve Length,Hair Length,Carrying,Accessory
0,,,,,,,,,,,,
1,upperBodyBlue,,lowerBodyBlack,hairBlack,footwearWhite,lowerBodyJeans,personalLess45,personalMale,upperBodyShortSleeve,hairShort,carryingMessengerBag,accessoryNothing
2,upperBodyRed,,lowerBodyGrey,hairBrown,footwearWhite,lowerBodyJeans,personalLess30,,upperBodyShortSleeve,hairShort,carryingNothing,NaN
3,upperBodyBlue,,lowerBodyBlack,hairBlack,footwearWhite,lowerBodyTrousers,personalLess30,personalMale,upperBodyShortSleeve,hairShort,carryingNothing,NaN
4,upperBodyBlack,Ellipsis,lowerBodyBlack,,,lowerBodyJeans,personalLess30,personalMale,upperBodyShortSleeve,hairShort,carryingNothing,NaN
5,upperBodyBlack,Ellipsis,lowerBodyBlue,,footwearBrown,lowerBodyJeans,personalLarger60,personalMale,upperBodyShortSleeve,,carryingNothing,accessoryNothing


In [454]:
data['Accessory'].isnull().sum() # = 88
#We will also drop the accessory column since it has a large chunk of unknown data
data = data.drop(['Accessory'],axis=1)

In [455]:
mask = data['Carrying'].str.startswith('carr',na=False)
data['Carrying'][~mask] = " "
data['Carrying'].value_counts()

carryingNothing         72
carryingMessengerBag    50
carryingOther           31
carryingBackpack        12
carryingSuitcase         4
carryingFolder           2
                         2
carryingPlasticBags      1
Name: Carrying, dtype: int64

In [475]:
data.loc[4]['Upper Body']

Ellipsis